In [ ]:
## Providing Training Data and Test Data were not matching the questions asking
## Code is for general use

import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import json
%matplotlib inline

products = pd.read_csv('amazon_baby.csv',dtype={'name': str, 'review': str, 'rating': float})
# products.shape
# products.head()


In [ ]:
products.head()
products['name'] = products['name'].astype('str')
products['review'] = products['review'].astype('str')
products = products.fillna({'review':''})

In [ ]:
def remove_punctuation(text):
    import string
    translator = str.maketrans(' ',' ', string.punctuation)
    return text.translate(translator) 


# def remove_punctuation2(text):
#     import string
#     return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)
products.shape

In [ ]:
# 3. We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment. 

products = products[products['rating'] != 3]
products.shape
products['sentiment'] = products['rating'].apply(lambda x: '+1' if x >3 else '-1')

In [ ]:
train_idx = pd.read_json('module-2-assignment-train-idx.json')
test_idx = pd.read_json('module-2-assignment-test-idx.json')

train_data = products.iloc[train_idx[0],:]
test_data = products.iloc[test_idx[0],:]
test_d2 = products.iloc[test_idx[0],:]

In [ ]:
train_data = train_data.fillna({'review':''})
train_data['review_clean'] = train_data['review'].apply(remove_punctuation)
train_data = train_data[train_data['rating'] != 3]
train_data.shape
train_data['sentiment'] = train_data['rating'].apply(lambda x: '+1' if x >3 else '-1')

test_data = test_data.fillna({'review':''})
test_data['review_clean'] = test_data['review'].apply(remove_punctuation)
test_data = test_data[test_data['rating'] != 3]
test_data.shape
test_data['sentiment'] = test_data['rating'].apply(lambda x: '+1' if x >3 else '-1')

# Build the word count vector for each review
## 6. We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
Compute the occurrences of the words in each review and collect them into a row vector.
Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.
The following cell uses CountVectorizer in scikit-learn. Notice the token_pattern argument in the constructor.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

# Train a sentiment classifier with logistic regression
### We will now use logistic regression to create a sentiment classifier on the training data.

## 7. Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.



In [ ]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

## 8. There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

### Quiz question: How many weights are >= 0?

In [83]:
positive_sentiment = (sentiment_model.coef_ >= 0).sum()

print(positive_sentiment)
### A: 68419

34518


# Making predictions with logistic regression
## 9. Now that a model is trained, we can make predictions on the test data. In this section, we will explore this in the context of 3 data points in the test data. Take the 11th, 12th, and 13th data points in the test data and save them to sample_test_data. The following cell extracts the three data points from the SFrame test_data and print their content:

In [ ]:
sample_test_data = test_data[10:13]
# test_data.tail(20)

In [ ]:
sample_test_data = test_data[10:13]
sample_test_data_2 = sample_test_data.reset_index()
sample_test_data_2['review'][0]


In [ ]:
sample_test_data_2['review'][1]

In [ ]:
score = sentiment_model.decision_function(test_matrix[8:13])*-1
# score = list(score[:,0])
# sentiment_model.score
score

In [ ]:
def class_prediction(score):
    
    predictions = []
    for i in score:
        if i > 0:
            prediction =1
        elif i <= 0:
            prediction =-1
        predictions.append(prediction)
    
    return predictions

In [ ]:
class_prediction(score)

In [ ]:
sentiment_model.predict(test_matrix[8:13])

Probability Predictions
12. Recall from the lectures that we can also calculate the probability predictions from the scores using:

$P(y_i = +1 | \mathbf{x}_i, \mathbf{w}) = \dfrac{1}{1+\exp{(-\mathbf{w}^\intercal h(\mathbf{x}_i))}}$

Using the scores calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range [0, 1].

<b>Checkpoint:</b> Make sure your probability predictions match the ones obtained from sentiment_model.

<font color =red><b>Quiz question:</b></font> Of the three data points in sample_test_data, which one (first, second, or third) has the lowest probability of being classified as a positive review?

<br>
Ans: <font color= red>3</font>

Quiz: Which of the following products are represented in the 20 most positive reviews?<br>
Ans: <font color= red>Britax Decathlon Convertible Car Seat, Tiffany</font>

Quiz: Which of the following products are represented in the 20 most negative reviews?

Ans: <font color= red>The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit
<br>
Peg-Perego Tatamia High Chair, White Latte
<br>
Safety 1st High-Def Digital Monitor</font>

In [ ]:
def calc_proba(scores):
    predictions = []
    for score in scores:
        prediction = 1/(1+math.exp(-score)) 
        predictions.append(prediction)
    return predictions

In [ ]:
calc_proba(score)

In [ ]:
test_data['prob'] =sentiment_model.predict_proba(test_matrix)[:,1]
test_data.sort_values(by='prob',ascending=False)

In [84]:
from operator import itemgetter
predictions = sentiment_model.predict_proba(test_matrix)
sorted(predictions,key=itemgetter(0),reverse=True)[0:20]

[array([  1.00000000e+00,   1.02893199e-19]),
 array([  1.00000000e+00,   2.48022198e-18]),
 array([  1.00000000e+00,   3.04595024e-17]),
 array([  1.00000000e+00,   2.36502267e-18]),
 array([  1.00000000e+00,   1.74324939e-23]),
 array([  1.00000000e+00,   1.17021179e-17]),
 array([  1.00000000e+00,   3.87098039e-24]),
 array([  1.00000000e+00,   3.37169787e-17]),
 array([  1.00000000e+00,   5.25578986e-19]),
 array([  1.00000000e+00,   6.59357792e-22]),
 array([  1.00000000e+00,   3.56986336e-20]),
 array([  1.00000000e+00,   2.88833165e-21]),
 array([  1.00000000e+00,   1.99984805e-17]),
 array([  1.00000000e+00,   2.47498149e-18]),
 array([  1.00000000e+00,   3.58030211e-18]),
 array([  1.00000000e+00,   1.16127055e-18]),
 array([  1.00000000e+00,   9.93258861e-17]),
 array([  1.00000000e+00,   1.10778863e-16]),
 array([  1.00000000e+00,   2.18297239e-16]),
 array([  1.00000000e+00,   2.78865988e-16])]

In [85]:
sorted(predictions,key=itemgetter(0),reverse=False)[0:20]

[array([  8.88178420e-16,   1.00000000e+00]),
 array([  1.99840144e-15,   1.00000000e+00]),
 array([  8.34887715e-14,   1.00000000e+00]),
 array([  1.25233157e-13,   1.00000000e+00]),
 array([  2.52908805e-13,   1.00000000e+00]),
 array([  3.45057316e-13,   1.00000000e+00]),
 array([  3.28943539e-11,   1.00000000e+00]),
 array([  3.29891670e-11,   1.00000000e+00]),
 array([  9.69757608e-11,   1.00000000e+00]),
 array([  1.00466302e-10,   1.00000000e+00]),
 array([  4.38062475e-10,   1.00000000e+00]),
 array([  4.87991203e-10,   1.00000000e+00]),
 array([  6.36955377e-10,   9.99999999e-01]),
 array([  6.48503251e-10,   9.99999999e-01]),
 array([  6.57814025e-10,   9.99999999e-01]),
 array([  6.59188926e-10,   9.99999999e-01]),
 array([  8.18745516e-10,   9.99999999e-01]),
 array([  1.09411880e-09,   9.99999999e-01]),
 array([  1.52924851e-09,   9.99999998e-01]),
 array([  1.74532766e-09,   9.99999998e-01])]

## 15. We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by

$accuracy=\frac{\textrm{# correctly classified examples}}{# total examples}$

This can be computed as follows:

- Step 1: Use the sentiment_model to compute class predictions.
- Step 2: Count the number of data points when the predicted class labels match the ground truth labels.
- Step 3: Divide the total number of correct predictions by the total number of data points in the dataset.

### Quiz Question: What is the accuracy of the sentiment_model on the test_data? Round your answer to 2 decimal places (e.g. 0.76).
0.91
### Quiz Question: Does a higher accuracy value on the training_data always imply that the classifier is better?
NO


In [ ]:
prediction = sentiment_model.predict(test_matrix)
test_data['prediction'] = prediction
test_data.head()

In [ ]:
## Quiz Question: What is the accuracy of the sentiment_model on the test_data? Round your answer to 2 decimal places 
## (e.g. 0.76).

accuracy = sentiment_model.score(test_matrix, test_data['sentiment'])
accuracy

## 16. There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected 20 words to work with. These are:

In [ ]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

In [ ]:
simple_model = LogisticRegression()
simple_model.fit(train_matrix_word_subset,train_data['sentiment'])

In [46]:
simple_model_coef_table = pd.DataFrame({'word':significant_words,
                                         'coefficient':simple_model.coef_[0,:]*-1})
# simple_model.coef_
# len(significant_words)

In [50]:
simple_model_coef_table.sort_values(by='coefficient' ,ascending=False)

,coefficient,word
6,1.673074,loves
5,1.509812,perfect
0,1.363690,love
2,1.192538,easy
1,0.944000,great
4,0.520186,little
7,0.503760,well
8,0.190909,able
3,0.085513,old
9,0.058855,car


### Quiz Question: Consider the coefficients of simple_model. How many of the 20 coefficients (corresponding to the 20 significant_words) are positive for the simple_model?

### Quiz Question: Are the positive words in the simple_model also positive words in the sentiment_model?

In [52]:
simple_model_coef_table[simple_model_coef_table['coefficient']>0].count()

coefficient    10
word           10
dtype: int64

In [54]:
(simple_model_coef_table['coefficient']>0).sum()

10

## 19. We will now compare the accuracy of the sentiment_model and the simple_model.

First, compute the classification accuracy of the sentiment_model on the train_data.

Now, compute the classification accuracy of the simple_model on the train_data.

### Quiz Question: Which model (sentiment_model or simple_model) has higher accuracy on the TRAINING set?

In [77]:
m_train_score = sentiment_model.score(train_matrix,train_data['sentiment'])
s_train_score = simple_model.score(train_matrix_word_subset,train_data['sentiment'])
print(m_train_score)
print(s_train_score)

0.968624452839
0.866822570007


## 20. Now, we will repeat this exercise on the test_data. Start by computing the classification accuracy of the sentiment_model on the test_data.

Next, compute the classification accuracy of the simple_model on the test_data.

### Quiz Question: Which model (sentiment_model or simple_model) has higher accuracy on the TEST set?



In [78]:
m_test_score = sentiment_model.score(test_matrix,test_data['sentiment'])
s_test_score = simple_model.score(test_matrix_word_subset,test_data['sentiment'])
print(m_test_score)
print(s_test_score)

0.932415406767
0.869360451164


## 21. It is quite common to use the majority class classifier as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

### Quiz Question: Enter the accuracy of the majority class classifier model on the test_data. Round your answer to two decimal places (e.g. 0.76).
0.84

### Quiz Question: Is the sentiment_model definitely better than the majority class classifier (the baseline)?
Yes

In [81]:
positive_sent = (test_data['sentiment']=='+1').sum()

positive_sent

28095

In [82]:
positive_sent/len(test_data)

0.84278257739380846